In [1]:
# register model

import json
import mlflow
import logging
import os
import dagshub
from dotenv import load_dotenv
load_dotenv() 
# Set up DagsHub credentials for MLflow tracking
dagshub_token = os.getenv("DAGSHUB_PAT")
if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)

# logging configuration
logger = logging.getLogger('model_registration')
logger.setLevel('DEBUG')

console_handler = logging.StreamHandler()
console_handler.setLevel('DEBUG')

file_handler = logging.FileHandler('model_registration_errors.log')
file_handler.setLevel('ERROR')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

def load_model_info(file_path: str) -> dict:
    """Load the model info from a JSON file."""
    try:
        with open(file_path, 'r') as file:
            model_info = json.load(file)
        logger.debug('Model info loaded from %s', file_path)
        return model_info
    except FileNotFoundError:
        logger.error('File not found: %s', file_path)
        raise
    except Exception as e:
        logger.error('Unexpected error occurred while loading the model info: %s', e)
        raise

def register_model(model_name: str, model_info: dict):
    """Register the model to the MLflow Model Registry."""
    try:
        model_uri = f"runs:/{model_info['run_id']}/{model_info['model_path']}"
        print(f"+++{model_uri}+++")
        # Register the model
        model_version = mlflow.register_model(model_uri, model_name)
        
        # Transition the model to "Staging" stage
        client = mlflow.tracking.MlflowClient()
        client.transition_model_version_stage(
            name=model_name,
            version=model_version.version,
            stage="Staging"
        )
        
        logger.debug(f'Model {model_name} version {model_version.version} registered and transitioned to Staging.')
    except Exception as e:
        logger.error('Error during model registration: %s', e)
        raise

def main():
    try:
        model_info_path = r'D:\yt-comment-sentiment-analysis2\experiment_info.json'
        model_info = load_model_info(model_info_path)
        
        model_name = "light-gbm-2.0"
        #register_model(model_name, model_info)
    except Exception as e:
        logger.error('Failed to complete the model registration process: %s', e)
        print(f"Error: {e}")

if __name__ == '__main__':
    main()

2026-02-03 14:09:45,320 - model_registration - DEBUG - Model info loaded from D:\yt-comment-sentiment-analysis2\experiment_info.json


# Mlflow 3 MOdel Alisa and Tags

In [4]:
import json
import mlflow
import logging
import os
import dagshub
from dotenv import load_dotenv
load_dotenv() 
# Set up DagsHub credentials for MLflow tracking
dagshub_token = os.getenv("DAGSHUB_PAT")
if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)

**registered Model Alias**

In [ ]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow")

# Assign the 'staging' alias to version 1
client.set_registered_model_alias(
    name="ligbm_model_v1", 
    alias="staging", 
    version="1"
)

**Load the model**

In [8]:
model = mlflow.lightgbm.load_model("models:/ligbm_model_v1@staging")

d:\yt-comment-sentiment-analysis2\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**registering new logged model and setting alias**

In [2]:
model_info_path = r'D:\yt-comment-sentiment-analysis2\experiment_info.json'
model_info = load_model_info(model_info_path)
model_info

2026-02-03 14:10:26,546 - model_registration - DEBUG - Model info loaded from D:\yt-comment-sentiment-analysis2\experiment_info.json


{'run_id': '214cc847cf284776934b74507f4bea5a', 'model_path': 'lgbm_model'}

In [ ]:
from mlflow import MlflowClient
client = MlflowClient(tracking_uri="https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow")
model_name = "light-gbm-2.0"
model_path = model_info["model_path"]
run_id = model_info["run_id"]
model_uri = f"runs:/{run_id}/{model_path}"
model_uri

'runs:/214cc847cf284776934b74507f4bea5a/lgbm_model'

In [ ]:
try:
    client.create_registered_model(model_name)
except Exception as e:
    print(e)
    #pass # Already exists

RESOURCE_ALREADY_EXISTS: Response: {'error_code': 'RESOURCE_ALREADY_EXISTS'}


In [11]:
mv = client.create_model_version(
    name=model_name,
    source=model_uri,
    run_id=run_id
)
# 3. Push to "Staging" by assigning an Alias (The MLflow 3+ Standard)
client.set_registered_model_alias(
    name=model_name,
    alias="production",
    version=mv.version
)
print(f"Model version {mv.version} is now registered and aliased to @staging")

2026/02/03 14:24:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: light-gbm-2.0, version 3


Model version 3 is now registered and aliased to @staging
